In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [127]:
#data = pd.read_excel("D:\MAGVI_500RT_RawData.xlsx")
url = "https://github.com/dklim77/MechLearnEdu/blob/master/MAGVI_500RT_RawData.xlsx?raw=true"
data = pd.read_excel(url)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1524 entries, 0 to 1523
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   No                  1524 non-null   int64  
 1   EvaporatingTemp     1524 non-null   float64
 2   CondensingTemp      1524 non-null   float64
 3   IGVOpening          1524 non-null   int64  
 4   HotGasFraction      1524 non-null   int64  
 5   RPM                 1524 non-null   int64  
 6   Capacity            1524 non-null   float64
 7   InputPower          1524 non-null   float64
 8   FirstTopHead        1524 non-null   float64
 9   SecondTopHead       1524 non-null   float64
 10  1stCompHead         1524 non-null   float64
 11  2ndCompHead         1524 non-null   float64
 12  TotalHead           1524 non-null   float64
 13  1stCompSurgeMargin  1524 non-null   float64
 14  2ndCompSurgeMargin  1524 non-null   float64
 15  VFR_1st             1524 non-null   float64
 16  VFR_2n

In [128]:
print(data.shape)
data[:5]

(1524, 17)


,No,EvaporatingTemp,CondensingTemp,IGVOpening,HotGasFraction,RPM,Capacity,InputPower,FirstTopHead,SecondTopHead,1stCompHead,2ndCompHead,TotalHead,1stCompSurgeMargin,2ndCompSurgeMargin,VFR_1st,VFR_2nd
0,2,2.0,19.0,80,0,15001,543.8,274.61,1664.8,1155.8,1072.7,136.3,1209.0,592.1,1019.5,0.7060,0.4532
1,3,2.0,19.0,60,0,15001,479.5,224.12,1585.0,1100.4,1016.9,192.4,1209.4,568.1,907.9,0.6183,0.4088
2,4,2.0,19.0,40,0,15001,389.1,186.39,1550.7,1077.4,979.7,234.1,1213.8,571.0,843.3,0.4995,0.3392
3,5,2.0,19.0,20,0,15001,254.7,146.55,1512.2,1053.8,931.7,300.5,1232.1,580.5,753.4,0.3253,0.2330
4,6,2.0,19.0,0,0,15001,58.3,94.05,1095.8,771.2,1060.0,292.3,1352.3,35.8,478.9,0.0760,0.0606


In [129]:
# 사본 사용 (원본 백업)
df = data.copy()
df[:3]

,No,EvaporatingTemp,CondensingTemp,IGVOpening,HotGasFraction,RPM,Capacity,InputPower,FirstTopHead,SecondTopHead,1stCompHead,2ndCompHead,TotalHead,1stCompSurgeMargin,2ndCompSurgeMargin,VFR_1st,VFR_2nd
0,2,2.0,19.0,80,0,15001,543.8,274.61,1664.8,1155.8,1072.7,136.3,1209.0,592.1,1019.5,0.7060,0.4532
1,3,2.0,19.0,60,0,15001,479.5,224.12,1585.0,1100.4,1016.9,192.4,1209.4,568.1,907.9,0.6183,0.4088
2,4,2.0,19.0,40,0,15001,389.1,186.39,1550.7,1077.4,979.7,234.1,1213.8,571.0,843.3,0.4995,0.3392


In [130]:
# 스케일링

# 표준 스케일링
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
df[["EvaporatingTemp_sc","CondensingTemp_sc","IGVOpening_sc","RPM_sc"]] = scale.fit_transform(df[["EvaporatingTemp","CondensingTemp","IGVOpening","RPM"]])

# Min Max 스케일링
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
df[["EvaporatingTemp_mmsc","CondensingTemp_mmsc","IGVOpening_mmsc","RPM_mmsc"]] = minmax.fit_transform(df[["EvaporatingTemp","CondensingTemp","IGVOpening","RPM"]])

df[:3]

,No,EvaporatingTemp,CondensingTemp,IGVOpening,HotGasFraction,RPM,Capacity,InputPower,FirstTopHead,SecondTopHead,1stCompHead,2ndCompHead,TotalHead,1stCompSurgeMargin,2ndCompSurgeMargin,VFR_1st,VFR_2nd,EvaporatingTemp_sc,CondensingTemp_sc,IGVOpening_sc,RPM_sc,EvaporatingTemp_mmsc,CondensingTemp_mmsc,IGVOpening_mmsc,RPM_mmsc
0,2,2.0,19.0,80,0,15001,543.8,274.61,1664.8,1155.8,1072.7,136.3,1209.0,592.1,1019.5,0.7060,0.4532,-1.622635,-1.391129,0.668366,1.095127,0.0,0.0,0.8,1.0
1,3,2.0,19.0,60,0,15001,479.5,224.12,1585.0,1100.4,1016.9,192.4,1209.4,568.1,907.9,0.6183,0.4088,-1.622635,-1.391129,-0.019406,1.095127,0.0,0.0,0.6,1.0
2,4,2.0,19.0,40,0,15001,389.1,186.39,1550.7,1077.4,979.7,234.1,1213.8,571.0,843.3,0.4995,0.3392,-1.622635,-1.391129,-0.707177,1.095127,0.0,0.0,0.4,1.0


In [131]:
#X = df[["EvaporatingTemp", "CondensingTemp", "IGVOpening", "RPM"]]
# 표준 스케일링 적용
X = df[["EvaporatingTemp_sc","CondensingTemp_sc","IGVOpening_sc","RPM_sc"]]

# Min Max 스케일링 적용
#X = df[["EvaporatingTemp_mmsc","CondensingTemp_mmsc","IGVOpening_mmsc","RPM_mmsc"]]

y = df[["Capacity", "InputPower", "FirstTopHead", "SecondTopHead", "1stCompHead", "2ndCompHead"]]
X[:5], y[:5]


(   EvaporatingTemp_sc  CondensingTemp_sc  IGVOpening_sc    RPM_sc
 0           -1.622635          -1.391129       0.668366  1.095127
 1           -1.622635          -1.391129      -0.019406  1.095127
 2           -1.622635          -1.391129      -0.707177  1.095127
 3           -1.622635          -1.391129      -1.394948  1.095127
 4           -1.622635          -1.391129      -2.082720  1.095127,
    Capacity  InputPower  FirstTopHead  SecondTopHead  1stCompHead  2ndCompHead
 0     543.8      274.61        1664.8         1155.8       1072.7        136.3
 1     479.5      224.12        1585.0         1100.4       1016.9        192.4
 2     389.1      186.39        1550.7         1077.4        979.7        234.1
 3     254.7      146.55        1512.2         1053.8        931.7        300.5
 4      58.3       94.05        1095.8          771.2       1060.0        292.3)

In [132]:
from sklearn.model_selection import train_test_split
from keras import layers
from keras import models
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [133]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
# monitor='val_loss' : validation set 의 loss 를 monitoring 한다
# mode='min' : performance measure를 최소화 시킴, default는 'auto'
# verbose=1 : 언제 keras 에서 training 을 멈추었는지를 화면에 출력

# mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='min', save_best_only=True)
# Early stopping 객체에 의해 트레이닝이 중지되었을 때 그 상태는 이전 모ㅔㄹ에 비해 일반적으로
# validation error가 높은 상이일 것이다. 따라사 중지된 상태가 최고의 모델이 아닐 것이다.
# 가장 validation performance가 좋은 모델을 저장하기 위해 keras에서는 ModelCheckpoint 객체가 존재

In [134]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=9)    ################
model = models.Sequential()
model.add(layers.Dense(50, activation='relu', input_shape = (4,)))    # 스케일링 할 것  x
#model.add(layers.BatchNormalization())
model.add(layers.Dense(50, activation='relu'))
# model.add(layers.Dense(150, activation='relu'))
# model.add(layers.Dense(100, activation='relu'))
#model.add(layers.BatchNormalization())
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(6, activation='linear'))    # activation='linear'  디폴트값임
model.summary()
# 안되면 레이어 추가할 것

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 50)                250       
_________________________________________________________________
dense_43 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_44 (Dense)             (None, 6)                 306       
Total params: 3,106
Trainable params: 3,106
Non-trainable params: 0
_________________________________________________________________


In [151]:
model.compile(optimizer= 'adam', loss = 'mean_squared_error')
history = model.fit(X_train, y_train, epochs=100, batch_size=20, verbose=1, validation_split = 0.2, callbacks=[early_stopping])    # early stopping   5정도 적용  5~10

Epoch 1/100
43/43 [==============================] - 1s 9ms/step - loss: 40.3416 - val_loss: 40.6465
Epoch 2/100
43/43 [==============================] - 0s 2ms/step - loss: 40.8882 - val_loss: 43.4106
Epoch 3/100
43/43 [==============================] - 0s 2ms/step - loss: 35.6636 - val_loss: 41.6686
Epoch 4/100
43/43 [==============================] - 0s 2ms/step - loss: 34.9860 - val_loss: 41.0483
Epoch 5/100
43/43 [==============================] - 0s 2ms/step - loss: 38.4444 - val_loss: 42.6115
Epoch 6/100
43/43 [==============================] - 0s 2ms/step - loss: 38.8640 - val_loss: 40.9114
Epoch 00006: early stopping


In [144]:
X_test.shape

(458, 4)

In [145]:
model.predict(X_test[:10])

array([[ 464.22577 ,  267.41492 , 1244.568   ,  866.9246  , 1176.6056  ,
         773.68884 ],
       [ 547.89276 ,  247.07257 , 1225.1542  ,  863.1039  , 1032.5818  ,
         520.07153 ],
       [ 150.50761 ,  183.08116 , 1521.5394  , 1052.407   , 1439.63    ,
        1014.4547  ],
       [ 482.17957 ,  297.3209  , 1401.2145  ,  982.2778  , 1269.0104  ,
         820.3218  ],
       [ 444.02994 ,  184.01286 , 1352.9452  ,  946.67175 ,  935.2694  ,
         203.09508 ],
       [ 634.81244 ,  361.66736 , 1737.7975  , 1208.111   , 1165.2314  ,
         257.07114 ],
       [  73.29917 ,  125.038605, 1107.8948  ,  776.2876  , 1062.2092  ,
         302.92325 ],
       [ 361.8861  ,  141.60109 , 1088.7869  ,  767.6657  ,  814.1637  ,
         264.92365 ],
       [ 479.18988 ,  230.52861 , 1345.729   ,  943.7669  , 1054.6139  ,
         506.26688 ],
       [ 294.7166  ,  200.23361 , 1508.948   , 1050.3988  , 1036.37    ,
         495.8389  ]], dtype=float32)

In [146]:
y_test.shape

(458, 6)

In [147]:
import pandas as pd
y_pred = model.predict(X_test)     # ravel 삭제

In [148]:
y_pred.shape

(458, 6)

In [149]:
y_test.shape

(458, 6)

In [150]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)     # 학습 부족    # 학습 과대 

0.9984053684943298